In [1]:
import os
import sys
import torchvision
import json
from transformers.tokenization_bert import BertTokenizer

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/media/moju/data/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')

In [5]:
import torch
import torch.nn as nn
import torchvision
from transformers.tokenization_bert import BertTokenizer
import config
from fgc_preprocess import FgcSerDataset, BertIdx
from torch.utils.data import DataLoader
from sup_model import BertSupSentClassification
from transformers import BertModel
from transformers.optimization import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm_notebook as tqdm
from fgc_support_retri.fgc_preprocess import HotpotDataset

In [8]:
with open(config.FGC_DEV) as f:
    items = json.load(f)

In [11]:
items[0]

{'DID': 'D004',
 'DTEXT': '元祐元年（1086年），宋哲宗即位，高太皇太后垂帘听政，回朝任礼部郎中、中书舍人、翰林学士，元祐四年（1089年）拜龙图阁学士，曾出任杭州、颍州等知州职务，官至礼部尚书。\n绍圣元年（1094年）被哲宗贬谪至惠州、儋州（海南岛）。\n元符三年（1100年），宋徽宗即位，向太后垂帘听政，下诏让苏轼北还。\n建中靖国元年（1101年），夏天因冷饮过度，下痢不止，又误服黄芪，结果病情恶化，「齿间出血如蚯蚓者无数」，七月二十八日于常州孙氏馆病卒，享年六十四岁。由弟苏辙归葬于郏县小峨眉山。南宋宋孝宗追赠谥号「文忠」。\n苏轼疲于应付新旧党争，遇事「如食内有蝇，吐之乃已」，苏轼既反对王安石比较急进的改革措施，也不同意旧党司马光尽废新法，所以虽然新党一直称苏轼为旧党，但其实他在新旧两党之间均受排斥，仕途坎坷，时常远贬外方，不过他在各地居官清正，为民兴利除弊，政绩颇善，口碑甚佳，杭州西湖的苏堤就是实证。',
 'QUESTIONS': [{'QID': 'D004Q01',
   'QTYPE': '基础题',
   'QTEXT': '苏东坡在宋哲宗时期,曾出任过哪些官职?',
   'ANSWER': [{'ATEXT': '礼部郎中、中书舍人、翰林学士、龙图阁学士、知州与礼部尚书',
     'ATOKEN': [{'text': '礼部郎中', 'start': 31},
      {'text': '中书舍人', 'start': 36},
      {'text': '翰林学士', 'start': 41},
      {'text': '龙图阁学士', 'start': 58},
      {'text': '知州', 'start': 73},
      {'text': '礼部尚书', 'start': 80}]}],
   'ASPAN': [{'text': '宋哲宗', 'start': 12, 'end': 15},
    {'text': '礼部郎中', 'start': 31, 'end': 35},
    {'text': '中书舍人', 'start': 36, 'end': 40},
    {'text': '翰林学士', 'start': 41, 'end': 45},
 

In [7]:
items[0]

{'DID': '5a8b57f25542995d1e6f1371',
 'DTEXT': '斯科特·德里克森（1966年7月16日出生）是美国导演、编剧和制片人。\n他住在加州洛杉矶。\n他最出名的是执导恐怖电影，如《险恶》、《艾米丽·罗斯的驱魔》、《让我们远离邪恶》，以及2016年惊奇电影宇宙版的《奇怪医生》\n爱德华·戴维斯·伍德（1924年10月10日至1978年12月10日）是美国电影制片人、演员、作家、制片人和导演。\n',
 'QID': '1',
 'QTEXT': '斯科特·德里克森和埃德·伍德是同一国籍的吗？',
 'QTYPE': '进阶题',
 'AMODE': 'Comparing-Members',
 'ANSWER': [{'ATEXT': '对', 'ATOKEN': {'text': '对', 'start': 0}},
  {'ATEXT': 'yes', 'ATOKEN': {'text': 'yes', 'start': 0}}],
 'SENTS': ['斯科特·德里克森（1966年7月16日出生）是美国导演、编剧和制片人。',
  '他住在加州洛杉矶。',
  '他最出名的是执导恐怖电影，如《险恶》、《艾米丽·罗斯的驱魔》、《让我们远离邪恶》，以及2016年惊奇电影宇宙版的《奇怪医生》',
  '爱德华·戴维斯·伍德（1924年10月10日至1978年12月10日）是美国电影制片人、演员、作家、制片人和导演。'],
 'SUP_EVIDENCE': [1, 0, 0, 1]}

In [8]:
bert_model_name = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
dataset = HotpotDataset(items, transform=BertIdx(tokenizer))

In [9]:
dataset[0].keys()

dict_keys(['DID', 'SID', 'QTEXT', 'sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])

In [10]:
len(dataset[0]['input_ids'])

56

In [11]:
len(dataset[1]['input_ids'])

34

In [12]:
len(dataset)

47988

In [13]:
dataset

In [15]:
# dataloader_train = DataLoader(dataset, batch_size=32, collate_fn=bert_collate)
# for batch in dataloader_train:
#     print(batch)
#     import pdb; pdb.set_trace()